# Parallel Image Processing with HPXPy

This notebook demonstrates parallel image processing operations using HPXPy.

**Topics covered:**
- Image normalization
- Channel operations with broadcasting
- Batch image statistics
- Parallel convolution concepts

## 1. Setup

In [1]:
import time
import numpy as np
import hpxpy as hpx

# Initialize HPX runtime
hpx.init()
print(f"HPXPy initialized with {hpx.num_threads()} threads")

HPXPy initialized with 12 threads


## 2. Creating Synthetic Images

We'll create synthetic grayscale and RGB images to demonstrate operations.

In [2]:
def create_gradient_image(height, width):
    """Create a horizontal gradient image."""
    x = np.linspace(0, 1, width, dtype=np.float64)
    return np.tile(x, (height, 1))

def create_checkerboard(height, width, square_size=8):
    """Create a checkerboard pattern."""
    x = np.arange(width) // square_size
    y = np.arange(height) // square_size
    return ((x[np.newaxis, :] + y[:, np.newaxis]) % 2).astype(np.float64)

def create_circle_image(height, width, center=None, radius=None):
    """Create an image with a circle."""
    if center is None:
        center = (height // 2, width // 2)
    if radius is None:
        radius = min(height, width) // 4
    
    y, x = np.ogrid[:height, :width]
    dist = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    return (dist <= radius).astype(np.float64)

# Create test images
H, W = 256, 256

gradient = create_gradient_image(H, W)
checkerboard = create_checkerboard(H, W, 32)
circle = create_circle_image(H, W)

print(f"Created images with shape: {gradient.shape}")
print(f"Gradient range: [{gradient.min():.2f}, {gradient.max():.2f}]")
print(f"Checkerboard unique values: {np.unique(checkerboard)}")
print(f"Circle pixels: {circle.sum():.0f} (inside circle)")

Created images with shape: (256, 256)
Gradient range: [0.00, 1.00]
Checkerboard unique values: [0. 1.]
Circle pixels: 12853 (inside circle)


## 3. Image Normalization

Normalize images using parallel operations.

In [3]:
def normalize_image_numpy(img):
    """Normalize image to [0, 1] range."""
    img_min = img.min()
    img_max = img.max()
    if img_max - img_min > 0:
        return (img - img_min) / (img_max - img_min)
    return img - img_min

def normalize_image_hpxpy(img):
    """Normalize image using HPXPy."""
    img_hpx = hpx.from_numpy(img) if not isinstance(img, hpx.ndarray) else img
    
    img_min = hpx.min(img_hpx)
    img_max = hpx.max(img_hpx)
    
    if img_max - img_min > 0:
        return (img_hpx - img_min) / (img_max - img_min)
    return img_hpx - img_min

# Test normalization
# Create an unnormalized image
unnorm = gradient * 200 + 50  # Range [50, 250]

norm_np = normalize_image_numpy(unnorm)
norm_hpx = normalize_image_hpxpy(unnorm)

print(f"Original range: [{unnorm.min():.0f}, {unnorm.max():.0f}]")
print(f"NumPy normalized: [{norm_np.min():.2f}, {norm_np.max():.2f}]")
print(f"HPXPy normalized: [{hpx.min(norm_hpx):.2f}, {hpx.max(norm_hpx):.2f}]")

Original range: [50, 250]
NumPy normalized: [0.00, 1.00]
HPXPy normalized: [0.00, 1.00]


## 4. RGB Channel Operations

Broadcasting enables efficient per-channel operations.

In [4]:
# Create a simple RGB image (H, W, 3)
def create_rgb_image(height, width):
    """Create synthetic RGB image."""
    img = np.zeros((height, width, 3), dtype=np.float64)
    img[:, :, 0] = create_gradient_image(height, width)      # Red: horizontal gradient
    img[:, :, 1] = create_gradient_image(height, width).T[:height, :width]  # Green: vertical
    img[:, :, 2] = create_circle_image(height, width) * 0.8  # Blue: circle
    return img

rgb = create_rgb_image(H, W)
print(f"RGB image shape: {rgb.shape}")

# Channel-wise statistics using broadcasting
rgb_hpx = hpx.from_numpy(rgb)

# Compute per-channel mean (would need axis support)
# For now, compute overall stats
print(f"\nChannel statistics:")
for c, name in enumerate(['Red', 'Green', 'Blue']):
    channel = hpx.from_numpy(rgb[:, :, c])
    print(f"  {name}: mean={hpx.mean(channel):.3f}, std={hpx.std(channel):.3f}")

RGB image shape: (256, 256, 3)

Channel statistics:
  Red: mean=0.500, std=0.290
  Green: mean=0.500, std=0.290
  Blue: mean=0.157, std=0.318


In [5]:
# Adjust brightness/contrast per channel
def adjust_brightness_contrast(img, brightness=0, contrast=1.0):
    """Adjust image brightness and contrast."""
    img_hpx = hpx.from_numpy(img) if not isinstance(img, hpx.ndarray) else img
    
    # Apply: out = (img - 0.5) * contrast + 0.5 + brightness
    adjusted = (img_hpx - 0.5) * contrast + 0.5 + brightness
    
    # Clip to [0, 1] - convert to numpy for clipping
    result = adjusted.to_numpy()
    return np.clip(result, 0, 1)

# Test adjustments
original = gradient.copy()
brightened = adjust_brightness_contrast(original, brightness=0.2)
contrasted = adjust_brightness_contrast(original, contrast=1.5)

print(f"Original mean: {original.mean():.3f}")
print(f"Brightened mean: {brightened.mean():.3f}")
print(f"Contrasted std: {contrasted.std():.3f} vs original: {original.std():.3f}")

Original mean: 0.500
Brightened mean: 0.680
Contrasted std: 0.373 vs original: 0.290


## 5. Batch Image Processing

Process multiple images in parallel.

In [6]:
# Create a batch of images
n_images = 100
batch_h, batch_w = 64, 64

# Create batch as (N, H, W) array
batch = np.random.rand(n_images, batch_h, batch_w).astype(np.float64)

print(f"Batch shape: {batch.shape}")
print(f"Total pixels: {batch.size:,}")

# Compute statistics for entire batch
batch_hpx = hpx.from_numpy(batch.reshape(-1))  # Flatten for reduction

print(f"\nBatch statistics:")
print(f"  Mean: {hpx.mean(batch_hpx):.4f}")
print(f"  Std:  {hpx.std(batch_hpx):.4f}")
print(f"  Min:  {hpx.min(batch_hpx):.4f}")
print(f"  Max:  {hpx.max(batch_hpx):.4f}")

Batch shape: (100, 64, 64)
Total pixels: 409,600

Batch statistics:
  Mean: 0.5004
  Std:  0.2886
  Min:  0.0000
  Max:  1.0000


In [7]:
def batch_normalize(images):
    """
    Normalize a batch of images.
    images: (N, H, W) array
    """
    # Global normalization across entire batch
    flat = hpx.from_numpy(images.reshape(-1))
    
    mean = hpx.mean(flat)
    std = hpx.std(flat)
    
    # Normalize
    normalized = (flat - mean) / (std + 1e-8)
    
    return normalized.to_numpy().reshape(images.shape)

# Normalize batch
start = time.perf_counter()
normalized_batch = batch_normalize(batch)
elapsed = (time.perf_counter() - start) * 1000

print(f"Normalized {n_images} images in {elapsed:.2f} ms")
print(f"Normalized mean: {normalized_batch.mean():.6f}")
print(f"Normalized std:  {normalized_batch.std():.6f}")

Normalized 100 images in 2.72 ms
Normalized mean: -0.000000
Normalized std:  1.000000


## 6. Performance Comparison

In [8]:
def benchmark_image_ops(n_images, img_size, warmup=3, repeats=5):
    """Benchmark image processing operations."""
    # Create batch
    batch = np.random.rand(n_images, img_size, img_size).astype(np.float64)
    flat = batch.reshape(-1)
    
    # Warmup
    for _ in range(warmup):
        _ = np.mean(flat)
        _ = hpx.mean(hpx.from_numpy(flat))
    
    # Time NumPy
    np_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        _ = np.mean(flat)
        _ = np.std(flat)
        normalized = (flat - flat.mean()) / (flat.std() + 1e-8)
        np_times.append(time.perf_counter() - start)
    
    # Time HPXPy
    hpx_times = []
    flat_hpx = hpx.from_numpy(flat)
    for _ in range(repeats):
        start = time.perf_counter()
        mean = hpx.mean(flat_hpx)
        std = hpx.std(flat_hpx)
        normalized = (flat_hpx - mean) / (std + 1e-8)
        hpx_times.append(time.perf_counter() - start)
    
    return min(np_times) * 1000, min(hpx_times) * 1000

print(f"Image Processing Performance (HPX threads: {hpx.num_threads()})")
print(f"{'Images':>8} {'Size':>8} {'Pixels':>12} {'NumPy (ms)':>12} {'HPXPy (ms)':>12} {'Speedup':>10}")
print("=" * 68)

configs = [
    (10, 256),
    (100, 128),
    (100, 256),
    (1000, 64),
]

for n_img, size in configs:
    pixels = n_img * size * size
    np_time, hpx_time = benchmark_image_ops(n_img, size)
    speedup = np_time / hpx_time if hpx_time > 0 else float('inf')
    print(f"{n_img:>8} {size:>8} {pixels:>12,} {np_time:>12.2f} {hpx_time:>12.2f} {speedup:>9.2f}x")

Image Processing Performance (HPX threads: 12)
  Images     Size       Pixels   NumPy (ms)   HPXPy (ms)    Speedup


      10      256      655,360         1.40         0.73      1.91x
     100      128    1,638,400         3.62         2.90      1.25x


     100      256    6,553,600        15.00         7.66      1.96x
    1000       64    4,096,000         9.54         7.50      1.27x


## 7. Cleanup

In [9]:
hpx.finalize()
print("HPX runtime finalized")

HPX runtime finalized


## Summary

HPXPy enables parallel image processing through:

1. **Parallel reductions** - `mean`, `std`, `min`, `max` on large images
2. **Element-wise operations** - Brightness, contrast, normalization
3. **Broadcasting** - Per-channel operations efficiently
4. **Batch processing** - Process many images simultaneously

### Future Extensions

With additional HPXPy features, we could add:
- Convolution filters (requires strided operations)
- FFT-based filtering (requires FFT)
- PCA/Eigenfaces (requires linear algebra)
- GPU-accelerated processing

### Further Reading

- [Digital Image Processing](https://en.wikipedia.org/wiki/Digital_image_processing)
- [Image Normalization](https://en.wikipedia.org/wiki/Normalization_(image_processing))
- [Eigenfaces](https://en.wikipedia.org/wiki/Eigenface)